In [1]:
# coding: utf-8
# Nicole Barberis, nbarberis@yahoo.com
# Objective: generate FFT features from raw data to use as additional input features for an LSTM project.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import fft
from scipy import fftpack, signal

In [3]:
### READ DATASET - original input data that you would like to create FFT features from
df = pd.read_csv('raw_external_data_final_02.csv', index_col=0)

In [4]:
# using dtypes to see the list of features
features = df.columns

In [5]:
index = df.index.values

In [6]:
df.head()

,Manu. Sol.,Auto Sol.,Manu. Ind.,Dig. Bldg. Sol.,Manu. Sol..1,Elec. Ind. Sol.,Energy Sol.,Elec. Ind. Sol..1,Manu. Sol,Manu. Sol..2,...,Europe_Govt_Outlays_Transpose,China_Govt_Outlays_Transpose,NTTYY,DTEGY,ORAN,AMZN,GOOG,ROC_NBS_Manufacturing_PMI,US_ISM_Manufactoring_PMI,EU_Markit_Manufacturing_PMI
date,,,,,,,,,,,,,,,,,,,,,
8/1/14,215351,35383,6690,10119,34796,17794,16177,43988,10862,36292,...,11856.82181,10203.70000,28.90,12.45,12.08,327.33,572.030154,51.1,58.1,51.8
9/1/14,226398,34622,6760,10172,39591,20497,16631,48152,11184,36487,...,11862.73190,14025.50000,28.16,12.46,11.90,330.31,580.295840,51.1,56.1,50.7
10/1/14,234778,37172,7448,10649,40188,21478,17277,50959,11725,37443,...,11855.63979,9909.80000,26.14,11.61,11.48,308.41,545.532236,50.8,57.9,50.3
11/1/14,204657,33411,6598,8217,35106,18511,14282,48956,9716,29699,...,11865.09593,12758.60000,25.10,12.94,13.01,318.69,541.491234,50.3,57.6,50.6
12/1/14,227327,37048,6634,8429,42098,22172,14575,50441,9611,38957,...,11849.72970,11482.53636,23.10,13.42,14.00,308.79,523.190399,50.1,55.1,50.1


In [7]:
len(features)

35

In [8]:
print(df.shape)

(57, 35)


In [9]:
#checking to see the last date
df.tail()

,Manu. Sol.,Auto Sol.,Manu. Ind.,Dig. Bldg. Sol.,Manu. Sol..1,Elec. Ind. Sol.,Energy Sol.,Elec. Ind. Sol..1,Manu. Sol,Manu. Sol..2,...,Europe_Govt_Outlays_Transpose,China_Govt_Outlays_Transpose,NTTYY,DTEGY,ORAN,AMZN,GOOG,ROC_NBS_Manufacturing_PMI,US_ISM_Manufactoring_PMI,EU_Markit_Manufacturing_PMI
date,,,,,,,,,,,,,,,,,,,,,
12/18/19,242072,39051,7599,9467,46340,25146,13707,54501,10382,41104,...,13076.62431,17431.95455,39.55,16.30,15.82,1559.44,1037.420519,49.4,54.3,51.8
1/19/19,227702,37637,7622,10754,39781,20329,16492,47917,11401,37461,...,13824.70546,18460.44000,41.20,15.91,15.11,1640.03,1072.264279,49.5,56.6,51.4
2/19/19,210846,36217,6922,9634,36332,17562,15254,42873,10938,34918,...,13827.50483,18534.28000,41.58,15.53,14.78,1626.94,1114.242123,49.2,54.2,50.5
3/19/19,243954,41032,7529,11163,44299,22560,16202,50652,12341,40932,...,13820.50640,25316.00000,42.12,16.42,15.26,1722.49,1179.197620,50.5,55.3,49.3
4/19/19,240665,40609,7365,12319,41708,21215,16443,46501,12769,43235,...,13840.10201,17038.00000,41.55,16.79,15.88,1866.20,1226.226190,50.1,52.8,47.5


In [10]:
df.iloc[:,-1].describe()

count    57.000000
mean     53.605263
std       2.926811
min      47.500000
25%      51.700000
50%      52.500000
75%      55.500000
max      60.600000
Name: EU_Markit_Manufacturing_PMI, dtype: float64

In [11]:
#df.iloc[:,-1].head()

## Adding the FFT functions.

In [12]:
def fourierExtrapolation(x, n_predict, n_harm):
    n = x.size
    # number of harmonics in model
    n_harm = n_harm                 
    t = np.arange(0, n)
    # find linear trend in x
    p = np.polyfit(t, x, 1)
    # detrended x
    x_notrend = x - p[0] * t        
    #x_freqdom = fft.fft(x_notrend)  
    # detrended x in frequency domain
    x_freqdom = fft.fft(x)
    # frequencies
    f = fft.fftfreq(n)              
    indexes = list(range(n))
    # sort indexes by frequency, lower -> higher
    indexes.sort(key = lambda i: np.absolute(f[i]))
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        # amplitude
        ampli = np.absolute(x_freqdom[i]) / n
        # phase
        phase = np.angle(x_freqdom[i])         
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
        
    forecast = restored_sig + p[0] * t
    return forecast[-n_predict:]

In [13]:
def getFFT(df_column):
  #sampling rate
  srate = 1
  # remove mean
  sd = df_column - np.mean(df_column)
  #apply FFT
  sd_fft = fftpack.fft(sd)
  #power spectrum
  sd_psd = np.abs(sd_fft/len(sd)) ** 2
  #freguencies
  fftfreq = fftpack.fftfreq(len(sd_fft), 1. / srate)
 
  return sd_fft, fftfreq, sd_psd

## Using the functions to create the new FFT features

In [14]:
result_list=[]
for i in range(len(df.columns)):
    temp = getFFT(df.iloc[:,i])
    result_list.append(temp)

In [15]:
# The output has 3 objects: sd_fft, fftfreq, sd_psd 
result_list[3][2]

array([3.66617446e-26, 3.37078453e+05, 6.99620287e+04, 2.50495839e+04,
       4.91039469e+04, 3.34137868e+05, 2.26515430e+04, 2.58309701e+03,
       3.31613456e+02, 2.49622170e+04, 1.73478815e+04, 7.09285783e+03,
       4.48207054e-01, 1.02505767e+03, 6.66002733e+03, 1.66164982e+03,
       8.88853734e+02, 1.73052241e+03, 1.76309109e+03, 6.27395845e+03,
       5.82919937e+04, 7.26157568e+03, 4.16159250e+03, 1.87338948e+04,
       2.97093294e+04, 3.21295366e+03, 1.63427330e+02, 1.37126702e+03,
       2.47352162e+03, 2.47352162e+03, 1.37126702e+03, 1.63427330e+02,
       3.21295366e+03, 2.97093294e+04, 1.87338948e+04, 4.16159250e+03,
       7.26157568e+03, 5.82919937e+04, 6.27395845e+03, 1.76309109e+03,
       1.73052241e+03, 8.88853734e+02, 1.66164982e+03, 6.66002733e+03,
       1.02505767e+03, 4.48207054e-01, 7.09285783e+03, 1.73478815e+04,
       2.49622170e+04, 3.31613456e+02, 2.58309701e+03, 2.26515430e+04,
       3.34137868e+05, 4.91039469e+04, 2.50495839e+04, 6.99620287e+04,
      

In [16]:
#get third elements, fft sd_psd, which is the Power Spectrum to use in project
raw_features_fft = [i[2] for i in result_list]

In [17]:
len(raw_features_fft[1])

57

In [18]:
raw_features_fft[3]

array([3.66617446e-26, 3.37078453e+05, 6.99620287e+04, 2.50495839e+04,
       4.91039469e+04, 3.34137868e+05, 2.26515430e+04, 2.58309701e+03,
       3.31613456e+02, 2.49622170e+04, 1.73478815e+04, 7.09285783e+03,
       4.48207054e-01, 1.02505767e+03, 6.66002733e+03, 1.66164982e+03,
       8.88853734e+02, 1.73052241e+03, 1.76309109e+03, 6.27395845e+03,
       5.82919937e+04, 7.26157568e+03, 4.16159250e+03, 1.87338948e+04,
       2.97093294e+04, 3.21295366e+03, 1.63427330e+02, 1.37126702e+03,
       2.47352162e+03, 2.47352162e+03, 1.37126702e+03, 1.63427330e+02,
       3.21295366e+03, 2.97093294e+04, 1.87338948e+04, 4.16159250e+03,
       7.26157568e+03, 5.82919937e+04, 6.27395845e+03, 1.76309109e+03,
       1.73052241e+03, 8.88853734e+02, 1.66164982e+03, 6.66002733e+03,
       1.02505767e+03, 4.48207054e-01, 7.09285783e+03, 1.73478815e+04,
       2.49622170e+04, 3.31613456e+02, 2.58309701e+03, 2.26515430e+04,
       3.34137868e+05, 4.91039469e+04, 2.50495839e+04, 6.99620287e+04,
      

In [19]:
# Convert to a dataframe
df4 = pd.DataFrame(raw_features_fft)

In [20]:
df4.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,9.385407e-24,7.143598e+07,5.379002e+06,3.155284e+06,2.858751e+06,1.253158e+07,1.341424e+06,69537.274059,499767.702797,4.478079e+06,...,6.181447e+06,4.478079e+06,499767.702797,69537.274059,1.341424e+06,1.253158e+07,2.858751e+06,3.155284e+06,5.379002e+06,7.143598e+07
1,7.185702e-24,9.326148e+05,1.116749e+05,1.011925e+05,4.884494e+04,4.143781e+04,9.299499e+04,4187.526392,1453.491277,7.612211e+04,...,1.698107e+05,7.612211e+04,1453.491277,4187.526392,9.299499e+04,4.143781e+04,4.884494e+04,1.011925e+05,1.116749e+05,9.326148e+05
2,4.302663e-26,7.012685e+04,1.145218e+04,1.047752e+04,1.676919e+04,3.972878e+04,1.518974e+03,215.832785,252.592721,8.865156e+03,...,8.483548e+03,8.865156e+03,252.592721,215.832785,1.518974e+03,3.972878e+04,1.676919e+04,1.047752e+04,1.145218e+04,7.012685e+04
3,3.666174e-26,3.370785e+05,6.996203e+04,2.504958e+04,4.910395e+04,3.341379e+05,2.265154e+04,2583.097009,331.613456,2.496222e+04,...,1.734788e+04,2.496222e+04,331.613456,2583.097009,2.265154e+04,3.341379e+05,4.910395e+04,2.504958e+04,6.996203e+04,3.370785e+05
4,1.629411e-26,1.496001e+06,2.638471e+05,1.707980e+05,2.012912e+05,4.173308e+05,3.863964e+03,10342.290932,14304.340490,1.159557e+05,...,2.010861e+05,1.159557e+05,14304.340490,10342.290932,3.863964e+03,4.173308e+05,2.012912e+05,1.707980e+05,2.638471e+05,1.496001e+06


In [21]:
# transpose
df5 = df4.T

In [22]:
df5.shape

(57, 35)

In [23]:
# reset columns names
df5.columns = features

In [24]:
df5.head()

,Manu. Sol.,Auto Sol.,Manu. Ind.,Dig. Bldg. Sol.,Manu. Sol..1,Elec. Ind. Sol.,Energy Sol.,Elec. Ind. Sol..1,Manu. Sol,Manu. Sol..2,...,Europe_Govt_Outlays_Transpose,China_Govt_Outlays_Transpose,NTTYY,DTEGY,ORAN,AMZN,GOOG,ROC_NBS_Manufacturing_PMI,US_ISM_Manufactoring_PMI,EU_Markit_Manufacturing_PMI
0,9.385407e-24,7.185702e-24,4.302663e-26,3.666174e-26,1.629411e-26,1.971587e-24,1.996028e-25,7.886349e-24,6.517643e-26,2.607057e-25,...,3.299557e-25,3.666174e-26,1.903561e-31,9.712046e-34,4.283012e-31,2.482703e-26,9.945135e-31,1.132813e-29,5.609678e-30,6.852819e-30
1,7.143598e+07,9.326148e+05,7.012685e+04,3.370785e+05,1.496001e+06,5.635262e+05,1.399336e+06,2.949530e+06,1.573830e+05,3.264586e+06,...,1.966565e+05,1.105383e+06,1.468360e+01,1.488443e-01,3.017437e-01,7.663134e+04,1.645592e+04,2.058153e-01,5.192600e+00,2.878203e+00
2,5.379002e+06,1.116749e+05,1.145218e+04,6.996203e+04,2.638471e+05,7.791781e+04,1.965792e+04,2.397533e+05,4.021257e+04,3.694522e+05,...,3.246189e+04,3.880741e+05,3.619901e+00,4.238805e-02,1.738066e-01,2.731749e+04,3.710202e+03,3.466938e-02,1.566623e-01,1.052804e+00
3,3.155284e+06,1.011925e+05,1.047752e+04,2.504958e+04,1.707980e+05,8.452103e+04,3.684805e+03,2.394202e+05,1.779128e+04,6.985039e+04,...,1.166861e+04,2.906577e+05,4.208763e-01,1.252187e-01,7.279692e-03,6.677722e+03,1.092177e+03,1.162728e-02,8.233070e-02,9.931214e-03
4,2.858751e+06,4.884494e+04,1.676919e+04,4.910395e+04,2.012912e+05,7.652175e+04,2.051688e+04,3.920352e+05,3.849860e+04,4.188653e+04,...,1.038369e+04,1.197797e+05,8.655319e-01,1.079352e-01,1.939331e-02,1.496546e+03,2.507855e+02,8.694509e-03,3.542925e-02,1.267227e-02


In [25]:
# Add index
df6 = df5.set_index(index)

In [26]:
df6.head()

,Manu. Sol.,Auto Sol.,Manu. Ind.,Dig. Bldg. Sol.,Manu. Sol..1,Elec. Ind. Sol.,Energy Sol.,Elec. Ind. Sol..1,Manu. Sol,Manu. Sol..2,...,Europe_Govt_Outlays_Transpose,China_Govt_Outlays_Transpose,NTTYY,DTEGY,ORAN,AMZN,GOOG,ROC_NBS_Manufacturing_PMI,US_ISM_Manufactoring_PMI,EU_Markit_Manufacturing_PMI
8/1/14,9.385407e-24,7.185702e-24,4.302663e-26,3.666174e-26,1.629411e-26,1.971587e-24,1.996028e-25,7.886349e-24,6.517643e-26,2.607057e-25,...,3.299557e-25,3.666174e-26,1.903561e-31,9.712046e-34,4.283012e-31,2.482703e-26,9.945135e-31,1.132813e-29,5.609678e-30,6.852819e-30
9/1/14,7.143598e+07,9.326148e+05,7.012685e+04,3.370785e+05,1.496001e+06,5.635262e+05,1.399336e+06,2.949530e+06,1.573830e+05,3.264586e+06,...,1.966565e+05,1.105383e+06,1.468360e+01,1.488443e-01,3.017437e-01,7.663134e+04,1.645592e+04,2.058153e-01,5.192600e+00,2.878203e+00
10/1/14,5.379002e+06,1.116749e+05,1.145218e+04,6.996203e+04,2.638471e+05,7.791781e+04,1.965792e+04,2.397533e+05,4.021257e+04,3.694522e+05,...,3.246189e+04,3.880741e+05,3.619901e+00,4.238805e-02,1.738066e-01,2.731749e+04,3.710202e+03,3.466938e-02,1.566623e-01,1.052804e+00
11/1/14,3.155284e+06,1.011925e+05,1.047752e+04,2.504958e+04,1.707980e+05,8.452103e+04,3.684805e+03,2.394202e+05,1.779128e+04,6.985039e+04,...,1.166861e+04,2.906577e+05,4.208763e-01,1.252187e-01,7.279692e-03,6.677722e+03,1.092177e+03,1.162728e-02,8.233070e-02,9.931214e-03
12/1/14,2.858751e+06,4.884494e+04,1.676919e+04,4.910395e+04,2.012912e+05,7.652175e+04,2.051688e+04,3.920352e+05,3.849860e+04,4.188653e+04,...,1.038369e+04,1.197797e+05,8.655319e-01,1.079352e-01,1.939331e-02,1.496546e+03,2.507855e+02,8.694509e-03,3.542925e-02,1.267227e-02


In [ ]:
# save file 
#df6.to_csv("raw_features_fft_sdpsd.csv", sep=',',index=True)
